In [1]:
from env import env
import flexpolyline as fp
import urllib.request
import json, sys
import pandas as pd
from bs4 import BeautifulSoup as bs

import random as rd


In [2]:
df = pd.read_csv('results/bike-results/result.csv')

### Get od pair and midpoint

In [3]:
# try later: https://geobgu.xyz/py/shapely.html

lat = df['station_lat'].to_list()
lng = df['station_lng'].to_list()

rd.seed(31)
x = rd.sample(range(0,2191),2)

origin_coord = (lat[x[0]],lng[x[0]])
origin = f'{origin_coord[0]},{origin_coord[1]}'

destination_coord = (lat[x[1]],lng[x[1]])
destination = f'{destination_coord[0]},{destination_coord[1]}'

#midpoint = ((origin_coord[0]+destination_coord[0])/2,(origin_coord[1]+destination_coord[1])/2)

print(origin)
print(destination)
#print(midpoint)

41.9106,-87.6494
41.76,-87.58
(41.835300000000004, -87.6147)


### Make request with od pair

In [4]:
"""for tomtom routing
query = f'{origin}:{destination}'
tomtomkey = env('TOM_TOM_KEY')
maxAlternatives = 0

url='https://api.tomtom.com/routing/1/calculateRoute/%s/json?&key=%s&sectionType=traffic&instructionsType=coded&maxAlternatives=%d&traffic=true&travelMode=car' % (query, tomtomkey, maxAlternatives)

tomtomResponse = urllib.request.urlopen(url).read().decode('utf-8')
tomtomResponse = json.loads(tomtomResponse)
print(url)"""

# co2 and fuel consumption link: https://developer.here.com/documentation/routing-api/dev_guide/topics/use-cases/calculate-fuel-consumption.html

query = f'destination={destination}&origin={origin}'
herekey = env('HERE_API_KEY')
fuel_consumption = 'transportMode=car&fuel[type]=diesel&fuel[freeflowspeedtable]=0,0,60,0.1,80,0.12&fuel[trafficspeedtable]=0,0,60,0.1,80,0.12&fuel[additionalconsumption]=1.8&fuel[ascent]=0.1'

url = 'https://router.hereapi.com/v8/routes?%s&%s&return=summary,polyline&apikey=%s' % (fuel_consumption,query,herekey)

hereResponse = urllib.request.urlopen(url).read().decode('utf-8')
hereResponse = json.loads(hereResponse)
print(url)

https://router.hereapi.com/v8/routes?transportMode=car&fuel[type]=diesel&fuel[freeflowspeedtable]=0,0,60,0.1,80,0.12&fuel[trafficspeedtable]=0,0,60,0.1,80,0.12&fuel[additionalconsumption]=1.8&fuel[ascent]=0.1&destination=41.76,-87.58&origin=41.9106,-87.6494&return=summary,polyline&apikey=RhbiMBCRVNDFplxIdn-yII6eZU9upCqvuChA2n7aSXo


### Print route on map with route midpoint

In [27]:
from haversine import haversine, Unit
import gmplot
import numpy as np

originCoord = origin.split(',')

output_map = gmplot.GoogleMapPlotter(originCoord[0],originCoord[1], 12)

colors = {
    'pedestrian':'blue',
    'bus':'yellow',
    'bicycle':'green'
}

lat_list = []
lng_list = []

first = True
for i in hereResponse['routes'][0]['sections']:
    
    #if first:
    #    lat_list.append(i['departure']['place']['location']['lat'])
    #    lng_list.append(i['departure']['place']['location']['lng'])
    #    
    #first = False
    
    try:
        polyline = fp.decode(i['polyline'])
        
        for coord in polyline:
            lat_list.append(coord[0])
            lng_list.append(coord[1])
    except:
        pass
        
    lat_list.append(i['arrival']['place']['location']['lat'])
    lng_list.append(i['arrival']['place']['location']['lng'])

output_map.plot(lat_list, lng_list, edge_width=3)

#tentativa 2: dist da rota com a api
middle = delta = hereResponse['routes'][0]['sections'][0]['summary']['length']/2
zip_list = list(zip(lat_list,lng_list))
before = after = None
dist = 0

for i in range(len(zip_list)-1):
    dist += haversine(zip_list[i],zip_list[i+1], unit=Unit.METERS)
    sub = abs(middle-dist)
    if sub < delta:
        delta = sub
        before = zip_list[i+1]
    if sub > delta:
        after = zip_list[i+1]
        break

midpoint = ((before[0]+after[0])/2,(before[1]+after[1])/2)

output_map.marker(before[0],before[1],color='blue')
output_map.marker(midpoint[0],midpoint[1],color='green')
output_map.marker(after[0],after[1],color='yellow')


""" tentantiva 1 - ponto médio da rota com midpoint
nearest = (0,0) # coord
dist = haversine(origin_coord,destination_coord) # distance

for la,lo in zip(lat_list,lng_list):
    value = haversine(midpoint,(la,lo))
    if value < dist: 
        dist = value
        nearest = (la,lo)

output_map.marker(midpoint[0],midpoint[1],color='blue')
output_map.marker(nearest[0],nearest[1],color='green')
"""

output_url = 'results/map_plot/Here_map.html'
output_map.draw(output_url)

# add the api key into html file
page = open(output_url)
soup = bs(page.read(),"html.parser")
src_key  = f"https://maps.googleapis.com/maps/api/js?key={env('GOOGLE_MAPS_KEY')}&libraries=visualization&sensor=true_or_false"
line = soup.find('script',type="text/javascript")
line['src'] = src_key

with open(output_url, "wb") as f_output:
    f_output.write(soup.prettify("utf-8"))  


In [ ]:
#def max_routing(start,end):